In [ ]:
# Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


## Player Count

* Display the total number of 


In [5]:
player_count = purchase_data["SN"].nunique()
player_count1 = pd.DataFrame({"Total Players":[ player_count]})
player_count1

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
unique_item = purchase_data["Item Name"].nunique()
unique_item

179

In [5]:
average_price = purchase_data["Price"].mean()
average_price

3.050987179487176

In [6]:
no_purchase = purchase_data["Purchase ID"].count()
no_purchase

780

In [7]:
total_price = "${:,.2f}".format(purchase_data["Price"].sum())
total_price

'$2,379.77'

In [8]:
purchase_df = pd.DataFrame({"Number of Unique Item":[unique_item],"Average Price":[average_price],"Number Of Purchase":[no_purchase],"Total Revenew":[total_price]})
purchase_df

,Number of Unique Item,Average Price,Number Of Purchase,Total Revenew
0,179,3.050987,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [23]:
#male = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")
gender_demo_df_ct = purchase_data.groupby(["Gender"]).agg({"SN" : "nunique"})
gender_demo_df_ct = gender_demo_df_ct.rename(columns={"SN":"Total Count"})
gender_demo_df_pc = gender_demo_df_ct.apply(lambda x: 100 * x / float(x.sum()))
gender_demo_df_pc = gender_demo_df_pc.rename(columns={"Total Count":"Percentage of Players"})
gender_demo_df = pd.concat([gender_demo_df_ct,gender_demo_df_pc], axis=1)
gender_demo_df = gender_demo_df.sort_values(by="Total Count", ascending=False)
#gender_demo_df = gender_demo_df[:].applymap("{:,.2f}%".format)
pd.options.display.float_format = "{:,.2f}%".format

gender_demo_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


    
## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [19]:
male2 = purchase_data.groupby(["Gender"]).value_count()
male2
male1 = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Purchase Count")
male1

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_count'

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

